https://github.com/lyang36/IMO25/blob/main/IMO25.pdf

- 生成 -> 验证 -> 修正: 的迭代循环
    - https://www.anthropic.com/engineering/building-effective-agents
        - Evaluator-optimizer
- 连续5次验证（对solution的验证）通过则算求解成功，连续10次重新生成（验证+修正）失败，则退出
    - 成功的关键在于初始解是否跟正确解有较大的 overlap；

### gemini 2.5 pro

- https://ai.google.dev/gemini-api/docs/thinking?hl=zh-cn
- 关于 thinking 部分的说明
    - https://blog.google/technology/google-deepmind/google-gemini-updates-io-2025/#developer-experience
    - Thought summaries
        - 2.5 Pro and Flash will now include thought summaries in the Gemini API and in Vertex AI. Thought summaries take the **model’s raw thoughts and organize them into a clear format with headers, key details and information about model actions**, like when they use tools.
        - We hope that with a more structured, streamlined format on the model’s thinking process, developers and users will find the interactions with Gemini models easier to understand and debug.
- role
    - system, user, model, user, model, ...
- ??
    - `"parts": [{"text": correction_prompt}, {"text": verify}]`
        - 刻画文本间的顺序，初衷是为了不同的模态
    - thinking budget: 似乎控制得不严格
        - step 2 的必要性存疑

### step 1 - 2

- markdown 格式
    - https://markdownlivepreview.com/
- instruction or requirements
    - Rigor is Paramount
    - Honesty About Completeness（prompt 层面避免所谓的 reward hacking）
    - Use TeX for All Mathematics
- output format
    - Summary 
        - a. Verdict => 后续做分支判断
        - b. Method Sketch
    - Detailed Solution => 后续做 verification
- Self-Correction Instruction

- 从 api 或者 context 的角度，step 2是接着 step 1 的对话接着回答
    - step1：system, user, model
    - step2: system, user, model, user, model
  
```python
self_improvement_prompt = """
You have an opportunity to improve your solution. Please review your solution carefully. Correct errors and fill justification gaps if any. Your second round of output should strictly follow the instructions in the system prompt.
"""
```

### step 3

- 角色扮演
- Core Instructions
    -  You must act as a verifier, NOT a solver.
    -  step-by-step check
- How to Handle Issues in the Solution
    - a. Critical Error:
    - b. Justification Gap:
-  Output Format
    -  a. Summary
        -  Final Verdict:
        -  List of Findings
            -  Location/issue: (Critical Error or Justification Gap).
    -  b. Detailed Verification Log

- 在 step 2 和 step 3 中间有一步 （条件控制，langgraph 的角度，条件边，conditional edge）
```python
check_complete_prompt = f"""
Is the following text claiming that the solution is complete?
==========================================================

{solution}

==========================================================

Response in exactly "yes" or "no". No other words.
    """
```
- 通过之后进入 verify
- extract_detailed_solution: 提取 Detailed Solution 之后的内容
- check correctness
```
Response in "yes" or "no". Is the following statement saying the solution is correct, or does not contain critical error or a major justification gap?
```
- if not yes
    - extract_detailed_solution: 提取 Detailed Verification 之后的内容